In [1]:
import os
%pwd
os.chdir("../")
%pwd


'd:\\Data Science\\END to END Proj\\BloodGroupClassifierBasedonFingerPrint'

In [2]:
%pip install dagshub
import dagshub
dagshub.init(repo_owner='gowtham-dd', repo_name='BloodGroupClassifierBasedonFingerPrint', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

  Using cached dagshub-0.6.2-py3-none-any.whl (261 kB)
  Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Using cached gql-3.5.3-py2.py3-none-any.whl (74 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached treelib-1.8.0-py3-none-any.whl (30 kB)
  Using cached pathvalidate-3.3.1-py3-none-any.whl (24 kB)
     -------------------------------------- 139.9/139.9 kB 1.7 MB/s eta 0:00:00
  Using cached dagshub_annotation_converter-0.1.11-py3-none-any.whl (35 kB)
  Using cached lxml-6.0.0-cp311-cp311-win_amd64.whl (4.0 MB)
     ---------------------------------------- 13.9/13.9 MB 2.0 MB/s eta 0:00:00
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.13.1-py3-none-any.whl (85 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
  Using cached graphql_core-3.2.6-py3-none-any.whl (203 kB)
  Using cached b


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c01f3979-7d79-49b1-9440-397d6459d8c0&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=21acd403ce3cfa154b3b0379998aa18323b367df808195f8bb531a60a1ffcefe




d:\Data Science\END to END Proj\BloodGroupClassifierBasedonFingerPrint\venv\Lib\site-packages\rich\live.py:256: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Accessing as gowtham-dd

Initialized MLflow to track repo "gowtham-dd/BloodGroupClassifierBasedonFingerPrint"

Repository gowtham-dd/BloodGroupClassifierBasedonFingerPrint initialized!

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path  # This is the folder containing image subdirectories
    model_path: Path
    metric_file_name: Path
    mlflow_uri: str
    batch_size: int
    target_metric: str
    all_params: dict


In [4]:
from src.BloodGroupClassifier.constant import *
from src.BloodGroupClassifier.utils.common import read_yaml,create_directories 
from BloodGroupClassifier.entity.config_entity import DataPreprocessingConfig

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.model_evaluation

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
        root_dir=Path(config.root_dir),
        test_data_path=Path(config.test_data_path),
        model_path=Path(config.model_path),
        metric_file_name=Path(config.metric_file_name),
        mlflow_uri=config.mlflow_uri,
        batch_size=params.batch_size,
        target_metric=params.target_metric,
        all_params=params
        )


In [6]:
import tensorflow as tf
import mlflow
from urllib.parse import urlparse
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from BloodGroupClassifier.utils.common import save_json
from BloodGroupClassifier import logger

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
        self.image_size = (64, 64)

    def _load_validation_generator(self):
        datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
        val_gen = datagen.flow_from_directory(
            self.config.test_data_path,
            target_size=self.image_size,
            batch_size=self.config.batch_size,
            class_mode='categorical',
            subset='validation',
            shuffle=False
        )
        return val_gen

    def _load_model(self):
        return tf.keras.models.load_model(self.config.model_path)

    def evaluate_model(self):
        model = self._load_model()
        val_gen = self._load_validation_generator()

        loss, accuracy = model.evaluate(val_gen)

        metrics = {
            "loss": float(loss),
            "accuracy": float(accuracy)
        }

        save_json(self.config.metric_file_name, metrics)
        logger.info(f"Evaluation metrics: {metrics}")
        return model, metrics

    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        try:
            with mlflow.start_run():
                model, metrics = self.evaluate_model()

                # Log hyperparameters
                mlflow.log_params(self.config.all_params)

                # Log metrics
                mlflow.log_metrics(metrics)

                # Log model
                if tracking_url_type_store != "file":
                    mlflow.tensorflow.log_model(
                        model,
                        artifact_path="cnn_model",
                        registered_model_name="Fingerprint_CNN_Model"
                    )
                else:
                    mlflow.log_artifacts(str(self.config.model_path), artifact_path="cnn_model")

                logger.info("Model evaluation and MLflow logging completed.")

        except Exception as e:
            logger.error(f"MLflow logging failed: {e}")
            if mlflow.active_run():
                mlflow.end_run(status="FAILED")
            raise


In [9]:

from BloodGroupClassifier import logger

try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluator = ModelEvaluation(config=eval_config)
    evaluator.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e


[2025-07-30 11:55:38,553: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-30 11:55:38,559: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-30 11:55:38,567: INFO: common: created directory at: artifacts]
[2025-07-30 11:55:38,570: INFO: common: created directory at: artifacts/model_evaluation]


Found 1197 images belonging to 8 classes.
38/38 [==============================] - 27s 706ms/step - loss: 1.2690 - accuracy: 0.8505
[2025-07-30 11:56:08,288: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2025-07-30 11:56:08,290: INFO: 2314356131: Evaluation metrics: {'loss': 1.269004464149475, 'accuracy': 0.8504594564437866}]


2025/07/30 11:56:09 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-07-30 11:56:11,620: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _update_step_xla while saving (showing 5 of 5). These functions will not be directly callable after loading.]
[2025-07-30 11:56:12,128: INFO: builder_impl: Assets written to: C:\Users\dell\AppData\Local\Temp\tmp0ih_g_ow\model\data\model\assets]


d:\Data Science\END to END Proj\BloodGroupClassifierBasedonFingerPrint\venv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'Fingerprint_CNN_Model'.
2025/07/30 11:57:11 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Fingerprint_CNN_Model, version 1


[2025-07-30 11:57:11,856: INFO: 2314356131: Model evaluation and MLflow logging completed.]


Created version '1' of model 'Fingerprint_CNN_Model'.
